In [1]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from scipy.spatial import ConvexHull
import folium

In [121]:
# importing data to notebook
data =pd.read_csv('../data/data_combined_v1.04.csv')
data.head()

,place_id,tripadvisor_id,fsq_id,name,business_status,types,price_level,ta_price_level,price_level_combined,rating,...,population_25_34,population_35_44,population_45_54,population_54_64,population_65+,resident_per_km2,brut_income_annual_per_person,district,district_cluster,combined_main_category_2
0,ChIJd6kh67ZkjEcRC1IQourQOyo,7159041.0,4de685281f6e9d90c7f5325c,Restaurant La Tentation,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",2.0,2.0,3.0,4.2,...,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3,European
1,ChIJX68uubZkjEcRKKxpZ7OrgqU,13322506.0,NaN,Café Restaurant Palma,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",NaN,NaN,4.0,3.7,...,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3,General
2,ChIJPcYsG_BljEcR4kTFqBJJoK4,6720272.0,NaN,Le Versailles,OPERATIONAL,"['restaurant', 'point_of_interest', 'food', 'e...",NaN,2.0,3.0,3.9,...,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3,European
3,ChIJV8uySFxljEcRn7XpIO3zZX0,NaN,NaN,Restaurant Les Jardins du Rhône,OPERATIONAL,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,NaN,NaN,NaN,...,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3,General
4,ChIJW8zU8rZkjEcR_Ke8t7c7hVc,NaN,NaN,Canteen De La Residence Gardens Du Rhône,OPERATIONAL,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,NaN,4.0,5.0,...,5163,5549,4924,4290,5625,16282,109266,Saint-Jean Charmilles,3,General


In [122]:
# coordinates for district clusters
df_cluster_centers = pd.read_csv('../data/data_cluster_centers_v1.02.csv').rename(columns={'district_area':'district_cluster'})
df_cluster_centers

,district_cluster,cluster_center_lat,cluster_center_lng
0,1,46.206097,6.128191
1,2,46.212504,6.129205
2,3,46.208559,6.120886
3,4,46.222885,6.127898
4,5,46.219508,6.137469
5,6,46.214862,6.131804
6,7,46.202012,6.137967
7,8,46.200854,6.130934
8,9,46.199478,6.134993
9,10,46.196314,6.139261


In [123]:
df = data[['district_cluster','geometry.location.lat','geometry.location.lng']]
df

,district_cluster,geometry.location.lat,geometry.location.lng
0,3,46.205232,6.114945
1,3,46.204931,6.114021
2,3,46.205752,6.114763
3,3,46.205248,6.115503
4,3,46.205387,6.115347
...,...,...,...
1563,14,46.207746,6.169275
1564,14,46.208511,6.169342
1565,36,46.192811,6.169873
1566,14,46.199892,6.174117


In [5]:
list_district_clusters = df['district_cluster'].unique().tolist()
list_district_clusters.sort()
list_district_clusters

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37]

In [16]:
def create_convexhull_polygon(map_object, list_of_points, layer_name, line_color, fill_color, weight, text): 

    # Since it is pointless to draw a convex hull polygon around less than 3 points check len of input
    if len(list_of_points) > 2:

        # Create the convex hull using scipy.spatial 
        form = [list_of_points[i] for i in ConvexHull(list_of_points).vertices]

        # Create feature group, add the polygon and add the feature group to the map 
        fg = folium.FeatureGroup(name=layer_name)
        fg.add_child(folium.vector_layers.Polygon(locations=form, color=line_color, fill_color=fill_color,
                                                  weight=weight, popup=(folium.Popup(text))))
        map_object.add_child(fg)

    return (map_object)

In [21]:
my_convexhull_map = folium.Map(location=[46.20494053262858, 6.142254182958967], zoom_start=13, max_zoom=14, min_zoom=12)

for i, row in df_cluster_points.head(10).iterrows():
    list_of_points = row['cluster_points']
    create_convexhull_polygon(my_convexhull_map, list_of_points, layer_name='Best Locations',
                        line_color='red', fill_color='lighblue', weight=5, text='French Restaurants')
    
for i, row in df_cluster_points.iterrows():
    list_of_points = row['cluster_points']
    create_convexhull_polygon(my_convexhull_map, list_of_points, layer_name='Worst Locations',
                        line_color='green', fill_color='lighblue', weight=5, text='French Restaurants')

In [76]:
map_object = folium.Map(location=[46.20494053262858, 6.142254182958967], zoom_start=13.6, tiles='cartodbpositron')

form = [a[i] for i in ConvexHull(a).vertices]

# Create feature group, add the polygon and add the feature group to the map 
fg = folium.FeatureGroup(name='layer_name')
fg.add_child(folium.vector_layers.Polygon(
    locations=form, 
    #color='red', 
    fill_color='red',
    weight=1,
    stroke=False,
    popup=(folium.Popup(f'{df_cluster_centers.iloc[36,0]}')),))
map_object.add_child(fg)

map_object